# DM-35005 - Checking new simulated data

We need two more targets to run the LVV-T2229 test case properly.  
Right now, we have an intra/extra-focus image at `20:28:18.74 -87:28:19.9` with PA=0º.

The [DM-35005] was created to hold up information about the creation of these new data.  
These new two targets need to be close (< 9 deg) and far from the target above (> 9 deg).  
This way, we can test it the mtaos will use the corrections from the initial target or not.   

Here are the coordinates of these two targets:

`20:37:32.54 -88:06:38.61` (close from the original target)  
`08:54:44.60 -83:25:33.66` (far from the original target)

[DM-35005]: https://jira.lsstcorp.org/browse/DM-35005

## Import Libraries

These are the libraries that we will use in this notebook. First, we need to use the [lsst.daf.butler.Butler].

[lsst.daf.butler.Butler]: https://pipelines.lsst.io/modules/lsst.obs.decam/using-obs-decam.html#local-repository-setup

In [21]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.coordinates import SkyCoord

import lsst.daf.butler  as dafButler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Defining a Local Butler Repository

Since I am trying to do this "the right way", I will try to use Butler for check on the data.  
For this, I will create a local repository using the documentation in the [Local Repository Setup] page.

[Local Repository Setup]: https://pipelines.lsst.io/modules/lsst.obs.decam/using-obs-decam.html#local-repository-setup

Use the following commands to enable the science pipeline within Nublado:
```
$ source /sdf/group/rubin/sw/w_latest/loadLSST.bash 
$ setup lsst_distrib
```

Then create an empty Butler repository:
```
$ cd ~/u/
$ butler create DM-35005
$ butler register-instrument DM-35005 lsst.obs.lsst.LsstComCam
```

Now we can copy the data to it. 

The data for `20:37:32.54 -88:06:38.61.   309.3855833 -88.1107250` were at:
* Intra focal fits files `/project/aos/closed_loop_high/comcam_111240/iter0/img/intra/`
* Extra focal fits files `/project/aos/closed_loop_high/comcam_111240/iter0/img/extra/`
* Infocus images `/project/aos/closed_loop_high/comcam_111240/infocus-repackaged/`

The data for `08:54:44.60 -83:25:33.66  133.6858333 -83.4260167` were at 
`/project/aos/closed_loop_high/comcam_111250/` same sub directories as above. 

I made a copy of these files in my user's home folder with the commands:
```
$ cp -R /sdf/group/rubin/ncsa-project/project/aos/closed_loop_high/comcam_111240 ~/u/DM-35005/
$ cp -R /sdf/group/rubin/ncsa-project/project/aos/closed_loop_high/comcam_111250 ~/u/DM-35005/
```

Where REPO is the name of the repository.

Ingest the data in the repository with the following command:
```
$ butler ingest-raws DM-35005 DM-35005/comcam_111240/iter0/img/intra/*.fits --transfer link
$ butler ingest-raws DM-35005 DM-35005/comcam_111240/iter0/img/extra/*.fits --transfer link
$ butler ingest-raws DM-35005 DM-35005/comcam_111240/infocus-repackaged/*[0-2].fits --transfer link
$ butler ingest-raws DM-35005 DM-35005/comcam_111250/iter0/img/intra/*.fits --transfer link
$ butler ingest-raws DM-35005 DM-35005/comcam_111250/iter0/img/extra/*.fits --transfer link
$ butler ingest-raws DM-35005 DM-35005/comcam_111250/infocus-repackaged/*.fits --transfer link
```

When executed in order, the first two lines above are accepted with the following message (or something very similaar):
```
$ butler ingest-raws DM-35005 DM-35005/comcam_111240/iter0/img/extra/*.fits --transfer link
lsst.ingest INFO: Successfully extracted metadata from 9 files with 0 failures
lsst.ingest INFO: Exposure LSSTComCam:CC_H_20211231_006001 ingested successfully
lsst.ingest INFO: Successfully processed data from 1 exposure with 0 failures from exposure registration and 0 failures from file ingest.
lsst.ingest INFO: Ingested 9 distinct Butler datasets
```

The third command fails with the following message:
```
$ butler ingest-raws DM-35005 DM-35005/comcam_111240/infocus-repackaged/*[0-2].fits --transfer link
lsst.ingest INFO: Successfully extracted metadata from 9 files with 0 failures
lsst.ingest WARNING: Failed to ingest the following for reason: A database constraint failure was triggered by inserting one or more datasets of type DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure) into collection 'LSSTComCam/raw/all'. This probably means a dataset with the same data ID and dataset type already exists, but it may also mean a dimension row is missing.
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S11.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S00.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S12.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S21.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S02.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S01.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S20.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S22.fits
lsst.ingest WARNING: - file:///sdf/data/rubin/u/bquint/DM-35005/comcam_111240/infocus-repackaged/CC_H_20211231_006001_R22_S10.fits
lsst.ingest INFO: Successfully processed data from 0 exposures with 0 failures from exposure registration and 1 failure from file ingest.
lsst.ingest INFO: Ingested 0 distinct Butler datasets
lsst.daf.butler.cli.utils ERROR: Caught an exception, details are in traceback:
Traceback (most recent call last):
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_base/g188f660963+e1f1daa193/python/lsst/obs/base/cli/cmd/commands.py", line 144, in ingest_raws
    script.ingestRaws(*args, **kwargs)
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_base/g188f660963+e1f1daa193/python/lsst/obs/base/script/ingestRaws.py", line 92, in ingestRaws
    ingester.run(
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/utils/g6f6f670c95+16108f2031/python/lsst/utils/timer.py", line 347, in timeMethod_wrapper
    res = func(self, *args, **keyArgs)
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_base/g188f660963+e1f1daa193/python/lsst/obs/base/ingest.py", line 1374, in run
    raise RuntimeError("Some failures encountered during ingestion")
RuntimeError: Some failures encountered during ingestion
```

The fourth command fails with the following message:
```
$ butler ingest-raws DM-35005 DM-35005/comcam_111250/iter0/img/intra/*.fits --transfer link
lsst.ingest INFO: Successfully extracted metadata from 9 files with 0 failures
lsst.ingest WARNING: Exposure LSSTComCam:CC_H_20211231_006002 could not be registered: Conflict in sync for table exposure on column(s) tracking_ra: 309.385583 != 133.685833, tracking_dec: -88.110725 != -83.426017.
lsst.ingest INFO: Successfully processed data from 0 exposures with 1 failure from exposure registration and 0 failures from file ingest.
lsst.ingest INFO: Ingested 0 distinct Butler datasets
lsst.daf.butler.cli.utils ERROR: Caught an exception, details are in traceback:
Traceback (most recent call last):
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_base/g188f660963+e1f1daa193/python/lsst/obs/base/cli/cmd/commands.py", line 144, in ingest_raws
    script.ingestRaws(*args, **kwargs)
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_base/g188f660963+e1f1daa193/python/lsst/obs/base/script/ingestRaws.py", line 92, in ingestRaws
    ingester.run(
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/utils/g6f6f670c95+16108f2031/python/lsst/utils/timer.py", line 347, in timeMethod_wrapper
    res = func(self, *args, **keyArgs)
  File "/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-4.1.0/Linux64/obs_base/g188f660963+e1f1daa193/python/lsst/obs/base/ingest.py", line 1374, in run
    raise RuntimeError("Some failures encountered during ingestion")
RuntimeError: Some failures encountered during ingestion
```

## Check the local repository

In [31]:
butler = dafButler.Butler(f"{os.getenv('HOME')}/u/DM-35005/")
registry = butler.registry

In [32]:
for c in sorted(registry.queryCollections()):
    print(c)

LSSTComCam/raw/all


In [33]:
for x in sorted(registry.queryDatasetTypes()):
    print(x)

DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure)


In [34]:
# You can also sub-select on specific properties of a data set
datasetRefs = registry.queryDatasets(datasetType='raw', collections="LSSTComCam/raw/all")

for i,ref in enumerate(datasetRefs):
    print(ref.dataId.full)

{band: 'g', instrument: 'LSSTComCam', detector: 0, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 8, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 5, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 4, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 3, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 7, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 1, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 6, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 2, physical_filter: 'g_01', exposure: 4021123106001}
{band: 'g', instrument: 'LSSTComCam', detector: 1, physical_filter: 'g_01', exposure: 40211

## Check Coordinates

In [35]:
original_field = SkyCoord("20:28:18.74 -87:28:19.9", unit=(u.hourangle, u.deg))
close_field = SkyCoord("20:37:32.54 -88:06:38.61", unit=(u.hourangle, u.deg))
far_field = SkyCoord("08:54:44.60 -83:25:33.66", unit=(u.hourangle, u.deg))

## Check Close Field

## Check Far Field